<a href="https://colab.research.google.com/github/nah90/CS777_PySparkLogReg/blob/main/MET_CS_777_Assignment_4_Horak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --ignore-installed -q pyspark==3.2.1

In [ ]:
import re
import sys
import numpy as np
from operator import add

from operator import add
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
n_words = 10000

In [ ]:
def buildArray(listOfIndices):
    
    returnVal = np.zeros(n_words)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    mysum = np.sum(returnVal)
    returnVal = np.divide(returnVal, mysum)
    return returnVal

In [ ]:
! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTrainingData.txt.bz2

# Use this code to read the data
courtFile = 'SmallTrainingData.txt.bz2'
corpus = sc.textFile(courtFile, 1)

In [ ]:
keyAndText = corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', str(x[1])).lower().split()))

topNWords = keyAndListOfWords.flatMap(lambda x: [(a,1) for a in x[1]]).reduceByKey(add).top(n_words, key = lambda x: x[1])

dictionary = sc.parallelize(range(n_words)).map(lambda x : (topNWords[x][0], x))
dict_data = dictionary.collectAsMap()
sc.broadcast(dict_data)


In [ ]:
#Task 1

def getPosition (inputWordList):
  for e in inputWordList:
    lookup = dict_data.get(e)
    if lookup != None:
      print("word: ", e, "-", "frequency: ", lookup)
    else:
      print("word: ", e, "-", "frequency: ",-1)

word_checks = ['applicant', 'and', 'attack', 'protein', 'car', 'court']#, 'humonginormous']

getPosition(word_checks) #Check word frequencies

word:  applicant - frequency:  346
word:  and - frequency:  2
word:  attack - frequency:  502
word:  protein - frequency:  3014
word:  car - frequency:  608
word:  court - frequency:  149


In [ ]:
#Task 2

allDictionaryWords = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1])).map(lambda x: (x[0], (x[1], dict_data.get(x[0]))) if x[0] in dict_data.keys() else None).filter(lambda x: x!=None)

allDocsAsNumpyArrays = allDictionaryWords.map(lambda x: (x[1][0], x[1][1])).groupByKey().map(lambda x: (x[0], buildArray(x[1])))

In [ ]:
traindata = allDocsAsNumpyArrays.map(lambda x: (1 if 'AU' in x[0] else 0, np.append(x[1],1)))
traindata.cache()
train_size = traindata.count()

print(traindata.take(10))

[(1, array([0.11813274, 0.06732296, 0.02349952, ..., 0.        , 0.        ,
       1.        ])), (1, array([0.11467201, 0.05658488, 0.03855784, ..., 0.        , 0.        ,
       1.        ])), (1, array([0.0742913 , 0.0400782 , 0.01075269, ..., 0.00195503, 0.00391007,
       1.        ])), (1, array([0.06669866, 0.04534549, 0.02855086, ..., 0.        , 0.        ,
       1.        ])), (1, array([8.68935554e-02, 4.41708907e-02, 2.53439537e-02, ...,
       3.62056481e-04, 0.00000000e+00, 1.00000000e+00])), (1, array([0.11010421, 0.0357952 , 0.01857725, ..., 0.        , 0.        ,
       1.        ])), (1, array([0.09835336, 0.0400534 , 0.02714731, ..., 0.        , 0.        ,
       1.        ])), (1, array([0.09986541, 0.04253028, 0.01588156, ..., 0.        , 0.        ,
       1.        ])), (1, array([0.07306122, 0.04683673, 0.01642857, ..., 0.        , 0.        ,
       1.        ])), (1, array([0.0866426 , 0.03936183, 0.02084546, ..., 0.        , 0.        ,
       1.        

In [ ]:
def LogisticRegression(traindata,
                       max_iteration,
                       learningRate,
                       regularization,
                       mini_batch_size,
                       train_size):

    #Initialization
    prev_cost = 0
    L_cost = []
    parameter_size = len(traindata.take(1)[0][1])
    np.random.seed(805) #My area code
    parameter_vector = np.zeros(parameter_size) #Initialize with zeros
    
    for i in range(max_iteration):

        bc_weights = parameter_vector

        mini_batch = traindata.sample(False, mini_batch_size / train_size, 1 + i) #Use mini-batch procedure

        res = mini_batch.treeAggregate((np.zeros(parameter_size), 0, 0), lambda x, y: (x[0] + y[1] * (-y[0] + (1 - 1/(np.exp(np.dot(y[1], bc_weights)) + 1))),\
                          x[1] + y[0] * (-(np.dot(y[1], bc_weights))) + np.log(1 + np.exp(np.dot(y[1], bc_weights))), x[2] + 1), add)

        cost =  res[1] + regularization * (np.square(parameter_vector).sum()) #Regularization l2
        
        #Calculate gradients
        gradient_derivative = (1.0 / res[2]) * res[0] + 2 * regularization * parameter_vector #Regularization l2
        
        #Sigmoid optimizer
        parameter_vector = parameter_vector - learningRate * gradient_derivative #Standard SGD
          

        print(i+1, " Cost =", cost)
        
        prev_cost = cost
        L_cost.append(cost)
        
    return parameter_vector, L_cost

pv, l_cost = LogisticRegression(traindata, 50, 29.75, 0.0002475, 256, train_size) #Smaller size for Small dataset than with Large

traindata.unpersist()

1  Cost = 163.5827346121474
2  Cost = 14.741999214118655
3  Cost = 71.95498767699834
4  Cost = 67.58006238660589
5  Cost = 50.81301651876855
6  Cost = 48.159638221243604
7  Cost = 68.36681563935919
8  Cost = 52.81667824339447
9  Cost = 68.75958544494453
10  Cost = 53.833464057226806
11  Cost = 89.11059535385998
12  Cost = 54.07028180651032
13  Cost = 17.966686568769383
14  Cost = 28.112476423714337
15  Cost = 31.034710928341607
16  Cost = 42.09277792108207
17  Cost = 35.84784833232595
18  Cost = 11.775173826555971
19  Cost = 20.42286225455132
20  Cost = 16.878270840592933
21  Cost = 28.87533767364011
22  Cost = 24.682084548814696
23  Cost = 24.902244165669824
24  Cost = 16.449268831321852
25  Cost = 20.812936976819376
26  Cost = 41.081035733463736
27  Cost = 31.995024883478873
28  Cost = 28.492870201310154
29  Cost = 31.526580588370962
30  Cost = 25.168660071951766
31  Cost = 16.957140465912524
32  Cost = 44.457852801014866
33  Cost = 38.2700092904375
34  Cost = 19.815690158445324
35  

PythonRDD[95] at RDD at PythonRDD.scala:53

In [ ]:
def getWord (inputPosList):
  swap_dict = dict((v,k) for k,v in dict_data.items())
  for e in inputPosList:
    lookup = swap_dict.get(e)
    print("position: ", e, "- word: ", lookup, "- coefficient:", pv[e])

pos_checks = pv.argsort()[-5:][::-1]

getWord(pos_checks) #Grab top 5 words with highest coefficient

position:  12 - word:  that - coefficient: 0.3422171569416813
position:  4 - word:  to - coefficient: 0.18433124081245328
position:  28 - word:  not - coefficient: 0.1395178633673223
position:  22 - word:  be - coefficient: 0.1297499559719862
position:  346 - word:  applicant - coefficient: 0.11949478899709662


In [ ]:
#Task 3

! wget -q https://storage.googleapis.com/met-cs-777-data/SmallTestingData.txt.bz2

# Use this code to read the data
courtFile2 = 'SmallTestingData.txt.bz2'
corpus2 = sc.textFile(courtFile2, 1)

In [ ]:
keyAndText2 = corpus2.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords2 = keyAndText2.map(lambda x : (str(x[0]), regex.sub(' ', str(x[1])).lower().split()))

In [ ]:
allDictionaryWords2 = keyAndListOfWords2.flatMap(lambda x: ((j, x[0]) for j in x[1])).map(lambda x: (x[0], (x[1], dict_data.get(x[0]))) if x[0] in dict_data.keys() else None).filter(lambda x: x!=None)

allDocsAsNumpyArrays2 = allDictionaryWords2.map(lambda x: (x[1][0], x[1][1])).groupByKey().map(lambda x: (x[0], buildArray(x[1])))

In [ ]:
testdata = allDocsAsNumpyArrays2.map(lambda x: (x[0], 1 if 'AU' in x[0] else 0, x[1]))

print(testdata.take(5))

[('AU11', 1, array([0.09323308, 0.04210526, 0.02706767, ..., 0.00075188, 0.        ,
       0.        ])), ('AU28', 1, array([0.10257817, 0.05485464, 0.0224904 , ..., 0.        , 0.        ,
       0.        ])), ('AU29', 1, array([0.09867374, 0.05145889, 0.01485411, ..., 0.        , 0.        ,
       0.        ])), ('AU31', 1, array([0.09686411, 0.03832753, 0.01393728, ..., 0.        , 0.        ,
       0.        ])), ('AU38', 1, array([0.09543246, 0.05889213, 0.03401361, ..., 0.00019436, 0.        ,
       0.        ]))]


In [ ]:
print("Bias: ", pv[-1]) #Negative bias, documents which return a 0 theta value are considered '0' rather than '1'

Bias:  -3.8838974964282182


In [ ]:
testresults = testdata.map(lambda x: (x[0], x[1], (np.dot(x[2],pv[:-1])))).map(lambda x: (x[0], x[1], (x[2] > 0).astype(int))) #Use '>' rather than '>=' as bias is negative
testresults.cache()

print(testresults.take(10))

TP = testresults.filter(lambda x: x[1] == 1 and x[2] == 1).count() #TP
FP = testresults.filter(lambda x: x[1] == 0 and x[2] == 1).count() #FP
FN = testresults.filter(lambda x: x[1] == 1 and x[2] == 0).count() #FN
TN = testresults.filter(lambda x: x[1] == 0 and x[2] == 0).count() #TN

F1 = TP/(TP + (0.50 * (FP + FN)))

print("TP: ", TP, "FP: ", FP, "FN: ", FN, "TN: ", TN)
print("F1 Score: ", F1)

[('AU11', 1, 1), ('AU28', 1, 1), ('AU29', 1, 1), ('AU31', 1, 1), ('AU38', 1, 0), ('AU67', 1, 1), ('AU77', 1, 1), ('AU78', 1, 1), ('AU87', 1, 1), ('AU88', 1, 1)]
TP:  18 FP:  0 FN:  2 TN:  20
F1 Score:  0.9473684210526315


In [ ]:
FPs = testresults.filter(lambda x: x[1] == 0 and x[2] == 1).map(lambda x: x[0]) #Look at FPs

keyAndText2.cache()

def falsePositives (falsePositiveExamples): #Find documents that triggered a FP
  example_count = 0
  for e in falsePositiveExamples:
      print('Document ID:', e, ' -  Document Text: ', keyAndText2.lookup(e))
      example_count += 1
  if example_count == 0:
      print('No False Positives!')

falsePositives(FPs.take(3))

No False Positives!


In [ ]:
sc.stop()
spark.stop()